# Train Model

In [6]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Connect to the SQLite database
conn = sqlite3.connect('Cleaned_Train_DB_80.db')
cursor = conn.cursor()

# Load data into df
cursor.execute("SELECT * FROM mockdata")
rows = cursor.fetchall()
columns = ['n', 'timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'label']
df = pd.DataFrame(rows, columns=columns)
conn.close()

# Preprocessing
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split data into features (X) and labels (y)
X = df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
y = df['label']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Sequential model
model = Sequential([
    Dense(64, activation='relu', input_shape=(6,)),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

model.save('Sequential.h5')

Epoch 1/10


C:\Users\bibek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5461 - loss: 2.1393 - val_accuracy: 0.6528 - val_loss: 0.7227
Epoch 2/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6348 - loss: 0.7697 - val_accuracy: 0.6675 - val_loss: 0.6806
Epoch 3/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6461 - loss: 0.7652 - val_accuracy: 0.6155 - val_loss: 0.9225
Epoch 4/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6436 - loss: 0.8194 - val_accuracy: 0.6786 - val_loss: 0.6549
Epoch 5/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6699 - loss: 0.6881 - val_accuracy: 0.6902 - val_loss: 0.6296
Epoch 6/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6833 - loss: 0.6473 - val_accuracy: 0.6781 - val_loss: 0.6179
Epoch 7/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6868 - loss: 0.6586 - val_accuracy: 0.6486 - val_loss: 0.6857
Epoch 8/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7048 - loss: 0.6402 - val_accuracy: 0.6649 - val_

# Test Model on new Data

In [8]:
from tensorflow.keras.models import load_model

new_data = {
    'ax': [-0.304688],
    'ay': [0.424316],
    'az': [-0.908691],
    'gx': [-6.793893],
    'gy': [-0.725191],
    'gz': [0.618321]
}
new_df = pd.DataFrame(new_data)
model = load_model('Sequential.h5')
predictions = model.predict(new_df)
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))
print(predicted_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
['normal']
